# About: マシン削除(Google Compute Engine)

Google Compute Engine上のマシンの削除をおこなうためのNotebook。

## Operation Note

*ここに経緯を記述*

# 動作環境の確認

このNotebookは、 [Google Python Client Library](https://github.com/google/google-api-python-client) を使ってマシンの削除を行います。

このNotebook環境にGoogle Python Client Libraryがインストールされている必要があります。インストールされていない場合は、以下のセル実行に失敗し、 `ImportError` となります。

In [1]:
from googleapiclient import discovery

もし、インストールされていない場合は、[Using the Python Client Library](https://cloud.google.com/compute/docs/tutorials/python-guide)を参考にインストールしてください。

In [2]:
#!pip2 install --upgrade google-api-python-client
#from googleapiclient import discovery

# 設定の定義

どのマシンを削除するか？を定義していく。

## Credentialの指定

Google Compute EngineにアクセスするためのCredentialを指示する。

- JSONフォーマットのService Account情報
- プロジェクトID

を用意しておく。

In [3]:
creds = '~/.keys/xxxxxxxx.json'
target_project_id = 'my_project_id'

computeサービスを取得しておく。

In [4]:
import os
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.from_stream(os.path.expanduser(creds))
compute = discovery.build('compute', 'v1', credentials=credentials)
compute

## 削除対象の指定

削除対象となるgroup名を定義する。

In [5]:
target_group = 'test-gce'

Bind対象への疎通状態を確認する。

In [6]:
!ansible -m ping {target_group}

XXX.XXX.XXX.59 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


ゾーンの定義を設定する。

In [7]:
target_zone = 'us-central1-f'

# マシンの削除

対象のIPアドレスを列挙する。

In [8]:
ping_stdout = !ansible -m ping {target_group}
import re
pattern = re.compile(r'^([0-9\.]+)\s*\|\s*SUCCESS\s+')
ping_stdout = filter(lambda l: pattern.match(l), ping_stdout)
ip_addresses = map(lambda l: pattern.match(l).group(1), ping_stdout)
ip_addresses

['XXX.XXX.XXX.59']

対象のIPアドレスに対応するインスタンス名を列挙する。

In [9]:
vm_descs = compute.instances().list(project=target_project_id, zone=target_zone).execute()['items']
vm_descs = filter(lambda i: filter(lambda interface: filter(lambda i: i['natIP'] in ip_addresses,
                                                            interface['accessConfigs']), i['networkInterfaces']), vm_descs)
vm_names = map(lambda i: i['name'], vm_descs)
vm_names

[u'test-gce']

In [10]:
assert(len(ip_addresses) == len(vm_names))

In [11]:
for vm_name in vm_names:
    resp = compute.instances().delete(project=target_project_id, zone=target_zone, instance=vm_name).execute()
resp

{u'id': u'649172682625639893',
 u'insertTime': u'2016-06-17T13:19:06.485-07:00',
 u'kind': u'compute#operation',
 u'name': u'operation-1466194746086-5357f11031271-ca798771-ffceb3aa',
 u'operationType': u'delete',
 u'progress': 0,
 u'selfLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f/operations/operation-1466194746086-5357f11031271-ca798771-ffceb3aa',
 u'status': u'PENDING',
 u'targetId': u'7960517509428916462',
 u'targetLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f/instances/test-gce',
 u'user': u'me@developer.gserviceaccount.com',
 u'zone': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f'}

# Inventoryの更新

Inventoryに、作成したマシンのIPアドレスを追加する。変更する前に、現在の内容をコピーしておく。

In [12]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp2lf3Uy'

In [13]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [14]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmp2lf3Uy/inventory-old	2016-06-18 05:19:14.221939266 +0900
+++ inventory	2016-06-18 05:19:19.965986220 +0900
@@ -3,6 +3,3 @@
 
 [test-vm]
 XXX.XXX.XXX.66
-
-[test-gce]
-XXX.XXX.XXX.59


Bind対象にpingが**通じない**かどうかを確認する。

In [15]:
!ansible -m ping {target_group}

 [WARNING]: provided hosts list is empty, only localhost is available



意図した結果となった。これで完了とする。

# 後始末

一時ディレクトリを削除する。

In [16]:
!rm -fr {work_dir}